In [2]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch.nn as nn

# Load data

In [3]:
df = pd.read_csv("data/cleandata.csv")

# Split data

In [12]:
class HeatDeseaseDataset(Dataset): 
    
    def __init__(self):
        self.xy = np.loadtxt("data/cleandata.csv", delimiter=",", dtype=np.float32, skiprows=1)
        self.x = torch.from_numpy(self.xy[:8000, 1:18])
        self.y = torch.from_numpy(self.xy[:8000, 22])

    def __len__(self):
        return 8000
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
        

In [13]:
class HeartDeseaseTestSet(Dataset):
    
    def __init__(self):
        self.xy = np.loadtxt("data/cleandata.csv", delimiter=",", dtype=np.float32, skiprows=1)
        self.x = torch.from_numpy(self.xy[8000:, 1:18])
        self.y = torch.from_numpy(self.xy[8000:, 22])

    def __len__(self):
        return self.xy.shape[0] - 8000
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# The logistic regession function

In [14]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return torch.sigmoid(out)

In [15]:
model = LogisticRegression(17, 1)

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

In [17]:
batch_size = 4

dataset = HeatDeseaseDataset()
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

testset = HeartDeseaseTestSet()
testloader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


#dataiter = iter(dataloader)
#data = dataiter.next()
len(dataset)

8000

In [18]:
iterator = 0
for epoch in range(10):
    print("g pa planté " + str(epoch))
    for i, (data, labels) in enumerate(dataloader):
        print("g pa planté " + str(i))
        data = data
        labels = labels

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(data)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iterator += 1

        if iterator % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for data, labels in testloader:
                # Load images to a Torch Variable
                data = data

                # Forward pass only to get logits/output
                outputs = model(data)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iterator, loss.item(), accuracy))

g pa planté 0
g pa planté 0


ValueError: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])) is deprecated. Please ensure they have the same size.